# example notebook to calculate a reasonable initial state for a MD simulation

# prerequisites
you need to install 
```pip install numpy scipy k3d```

# coordinates for a water molecule found on the web

ATOM      1  OH  OSP3    1       4.013   0.831  -9.083  1.00  0.00              
ATOM      2 1HH  OSP3    1       4.941   0.844  -8.837  1.00  0.00              
ATOM      3 2HH  OSP3    1       3.750  -0.068  -9.293  1.00  0.00       

In [1]:
coord=[[4.013,   0.831 , -9.083],[4.941,   0.844 , -8.837],[3.750,  -0.068,  -9.293]]

In [2]:
import numpy as np

In [3]:
coord=np.array(coord)

In [4]:
coord=coord-coord.mean(axis=0)

In [5]:
coord

array([[-0.22166667,  0.29533333, -0.012     ],
       [ 0.70633333,  0.30833333,  0.234     ],
       [-0.48466667, -0.60366667, -0.222     ]])

I check that is actually water

In [6]:
np.sum((coord[0]-coord[1])**2)**0.5

0.9601400939446285

In [7]:
np.sum((coord[2]-coord[1])**2)**0.5

1.5678523527424384

In [8]:
np.sum((coord[0]-coord[2])**2)**0.5

0.9599322892787799

In [9]:
import k3d

In [10]:
pl=k3d.plot()
pl += k3d.points(np.array(coord,dtype='float32'))
pl.display()

Output()

In [11]:
from scipy.spatial.transform import Rotation as R


Rember that with the question mark operator you can always get some documentation for any function. With two question mark you get the source code if available

In [12]:
R.create_group?

In [13]:
R1 = R.create_group('C4',axis='Z')
R2 = R.create_group('C2',axis='X')

einsum is my favourite numpy functions. It performs sums over repeated indexes, the position of the indexes is the same of the python indexes position. Like einstein notation on a piece of paper!
Here I apply the random matrix rotation of the molecule to the coordinate index. I try few random matrixes till the plot few cells below is pretty. R.random() is a random rotation matrix

In [14]:
l=2.0
coordr = np.einsum('ij,jm->im',coord,R.random().as_matrix())
print(coordr)
coords = np.einsum('ix,jxy->jiy',coordr+np.array([l,l,l]),R1.as_matrix())
coords2 = np.einsum('ix,jxy->jiy',coordr+np.array([l,-l,-l]),R1.as_matrix())

[[ 0.04935694  0.23905666 -0.27734039]
 [-0.70076153  0.23241062  0.32194844]
 [ 0.65140459 -0.47146728 -0.04460805]]


In [15]:
print(coords)
print("")
print(coords2)

[[[ 2.04935694  2.23905666  1.72265961]
  [ 1.29923847  2.23241062  2.32194844]
  [ 2.65140459  1.52853272  1.95539195]]

 [[ 2.23905666 -2.04935694  1.72265961]
  [ 2.23241062 -1.29923847  2.32194844]
  [ 1.52853272 -2.65140459  1.95539195]]

 [[-2.04935694 -2.23905666  1.72265961]
  [-1.29923847 -2.23241062  2.32194844]
  [-2.65140459 -1.52853272  1.95539195]]

 [[-2.23905666  2.04935694  1.72265961]
  [-2.23241062  1.29923847  2.32194844]
  [-1.52853272  2.65140459  1.95539195]]]

[[[ 2.04935694 -1.76094334 -2.27734039]
  [ 1.29923847 -1.76758938 -1.67805156]
  [ 2.65140459 -2.47146728 -2.04460805]]

 [[-1.76094334 -2.04935694 -2.27734039]
  [-1.76758938 -1.29923847 -1.67805156]
  [-2.47146728 -2.65140459 -2.04460805]]

 [[-2.04935694  1.76094334 -2.27734039]
  [-1.29923847  1.76758938 -1.67805156]
  [-2.65140459  2.47146728 -2.04460805]]

 [[ 1.76094334  2.04935694 -2.27734039]
  [ 1.76758938  1.29923847 -1.67805156]
  [ 2.47146728  2.65140459 -2.04460805]]]


In [16]:
pl=k3d.plot()
pl += k3d.points(np.array(coords,dtype='float32'))
pl += k3d.points(np.array(coords2,dtype='float32'))
pl.display()

Output()

I like the position of the atoms very much, so I prepare the input for QE. I convert to Bohr because I like it

In [17]:
coordf=np.concatenate([coords,coords2])

first index is of the molecule, second of the atom in the molecule. The third is the coordinate index

In [18]:
coordf.shape

(8, 3, 3)

In [19]:
types=['O', 'H', 'H']
for imol in range(coordf.shape[0]):
    for itype in range(coordf.shape[1]):
        x,y,z=coordf[imol][itype]/0.529
        print(f'{types[itype]} {x} {y} {z}')

O 3.8740206819121705 4.23262129178307 3.256445383871692
H 2.456027346400072 4.220057885133582 4.389316522182073
H 5.012106981139552 2.8894758325351435 3.696393103398031
O 4.232621291783071 -3.87402068191217 3.256445383871692
H 4.220057885133583 -2.4560273464000715 4.389316522182073
H 2.889475832535145 -5.012106981139551 3.696393103398031
O -3.87402068191217 -4.232621291783071 3.256445383871692
H -2.4560273464000715 -4.220057885133582 4.389316522182073
H -5.012106981139552 -2.889475832535144 3.696393103398031
O -4.232621291783071 3.87402068191217 3.256445383871692
H -4.220057885133583 2.4560273464000715 4.389316522182073
H -2.889475832535145 5.012106981139551 3.696393103398031
O 3.8740206819121705 -3.3288153811847936 -4.304991289096172
H 2.456027346400072 -3.3413787878342815 -3.1721201507857915
H 5.012106981139552 -4.67196084043272 -3.865043569569832
O -3.3288153811847927 -3.8740206819121714 -4.304991289096172
H -3.341378787834281 -2.456027346400073 -3.1721201507857915
H -4.671960840432

copy and paste the output of the cell above in your input file! Then after few cycles of relaxation the input will be ready for running a molecular dynamics simulation!